[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.leafmap.org/lab/index.html?path=notebooks/111_zarr.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/leafmap/blob/master/docs/notebooks/111_zarr.ipynb)

**Visualizing Zarr Data**

This notebook demonstrates how to visualize Zarr datasets on interactive maps using leafmap. [Zarr](https://zarr.dev/) is a cloud-optimized format for storing large N-dimensional arrays, making it ideal for geospatial and scientific data.

The `add_zarr` method uses [titiler-xarray](https://developmentseed.org/titiler/packages/xarray) for dynamic tile serving from Zarr datasets.

**References:**
- [EOPF Sentinel Zarr Explorer](https://explorer.eopf.copernicus.eu/)
- [titiler-eopf](https://github.com/EOPF-Explorer/titiler-eopf)
- [EOPF 101](https://eopf-toolkit.github.io/eopf-101/)
- [Zarr Visualization Report](https://nasa-impact.github.io/zarr-visualization-report/)

## Prerequisites

To visualize Zarr data, you need a TiTiler endpoint with titiler-xarray support. The default TiTiler endpoint does NOT support Zarr/xarray datasets.

You have two options:

1. **Start a local titiler-xarray server** (recommended for testing)
2. **Use a remote titiler-xarray endpoint** (if available)

### Install required packages

In [ ]:
# %pip install -U leafmap "titiler.xarray[full]" uvicorn xarray zarr fsspec aiohttp

In [ ]:
import leafmap

## Option 1: Start a Local TiTiler-XArray Server

The easiest way to get started is to run a local titiler-xarray server. This requires the `titiler.xarray` package to be installed.

In [ ]:
# Start the local titiler-xarray server
# This will return the endpoint URL
endpoint = leafmap.run_titiler_xarray()
print(f"TiTiler-XArray endpoint: {endpoint}")

## Working with Zarr Metadata

Before visualizing, let's explore the Zarr dataset using the helper functions.

### Get Available Variables

Use the `zarr_variables` function to list all variables in a Zarr dataset.

In [ ]:
# GPCP Precipitation dataset - a publicly available Zarr dataset
url = "https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/gpcp-feedstock/gpcp.zarr"

# Get available variables
variables = leafmap.zarr_variables(url, titiler_endpoint=endpoint)
print(f"Available variables: {variables}")

### Get Dataset Information

Use the `zarr_info` function to get detailed information about a Zarr dataset.

In [ ]:
# Get info requires specifying a variable for titiler-xarray
info = leafmap.zarr_info(url, variable="precip", titiler_endpoint=endpoint)
print(f"Bounds: {info.get('bounds')}")
print(f"CRS: {info.get('crs')}")

### Get Dataset Bounds

Use the `zarr_bounds` function to get the geographic bounds of a Zarr dataset.

In [ ]:
bounds = leafmap.zarr_bounds(url, variable="precip", titiler_endpoint=endpoint)
print(f"Bounds (minx, miny, maxx, maxy): {bounds}")

## Visualizing Zarr Data with ipyleaflet backend

The `add_zarr` method allows you to add Zarr datasets to the map. It requires:
- A URL to the Zarr dataset
- A variable name for multi-variable datasets
- A titiler endpoint with xarray support

**Note:** For datasets with a time dimension, the `time_index` parameter specifies which time step to display (default is 0, the first time step).

In [ ]:
m = leafmap.Map(center=[0, 0], zoom=1)
m.add_zarr(
    url,
    variable="precip",
    name="Precipitation",
    colormap_name="blues",
    rescale="0,20",
    titiler_endpoint=endpoint,
    time_index=0,  # Display first time step (default)
)
m

### Displaying Different Time Steps

You can change the `time_index` parameter to visualize different time steps.

In [ ]:
m = leafmap.Map(center=[0, 0], zoom=1)
m.add_zarr(
    url,
    variable="precip",
    name="Precipitation (time=100)",
    colormap_name="viridis",
    rescale="0,20",
    titiler_endpoint=endpoint,
    time_index=100,  # Display 100th time step
)
m

## Visualizing Zarr Data with MapLibre backend

The `add_zarr` method is also available in the MapLibre backend.

In [ ]:
import leafmap.maplibregl as leafmap

In [ ]:
m = leafmap.Map(center=[0, 0], zoom=1)
m.add_zarr(
    url,
    variable="precip",
    name="Precipitation",
    colormap_name="viridis",
    rescale="0,20",
    titiler_endpoint=endpoint,
    time_index=0,
)
m

## Using xarray to Explore Zarr Data

You can also use xarray directly to explore Zarr datasets before visualization.

In [ ]:
import xarray as xr

# Open the GPCP precipitation dataset
url = "https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/gpcp-feedstock/gpcp.zarr"
ds = xr.open_zarr(url)
ds

In [ ]:
# List all data variables
print("Data variables:")
for var in ds.data_vars:
    print(f"  - {var}: {ds[var].dims}")

In [ ]:
# Get information about a specific variable
ds["precip"]

In [ ]:
# Show available time steps
print(f"Number of time steps: {len(ds.time)}")
print(f"First time: {ds.time.values[0]}")
print(f"Last time: {ds.time.values[-1]}")

## Option 2: Using a Remote TiTiler-XArray Endpoint

If you have access to a remote titiler-xarray endpoint (e.g., from titiler-eopf), you can use it directly:

```python
# Set the endpoint URL
endpoint = "https://your-titiler-xarray-endpoint.com"

m = leafmap.Map()
m.add_zarr(
    url="https://example.com/data.zarr",
    variable="temperature",
    titiler_endpoint=endpoint,
)
m
```

You can also set the endpoint as an environment variable:

```python
import os
os.environ["TITILER_XARRAY_ENDPOINT"] = "https://your-titiler-xarray-endpoint.com"
```

## Summary

Key functions for working with Zarr data in leafmap:

- `leafmap.run_titiler_xarray()` - Start a local titiler-xarray server
- `map.add_zarr()` - Add a Zarr dataset to the map
  - `time_index` parameter specifies which time step to display (default: 0)
- `leafmap.zarr_variables()` - Get list of variables in a Zarr dataset
- `leafmap.zarr_info()` - Get metadata about a Zarr dataset
- `leafmap.zarr_bounds()` - Get geographic bounds of a Zarr dataset
- `leafmap.zarr_statistics()` - Get statistics for a Zarr variable

### Public Zarr Datasets for Testing

Here are some publicly available Zarr datasets you can use:

- **GPCP Precipitation**: `https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/gpcp-feedstock/gpcp.zarr`